**Notebook preparation and importing libraries**

In [1]:
from pathlib import Path
from hvac import Quantity
from hvac.fluids import Fluid, HumidAir, CoolPropWarning
from hvac.vapor_compression import VariableSpeedCompressor
from hvac.vapor_compression.machine import SingleStageVaporCompressionMachine
from hvac.heat_transfer.heat_exchanger.fin_tube import PlainFinTubeCounterFlowAirEvaporator
from hvac.heat_transfer.heat_exchanger.fin_tube import PlainFinTubeCounterFlowAirCondenser

In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity

# Rating of a Single-Stage Vapor Compression Machine

A single-stage vapor compression machine is composed of a compressor, an evaporator, a condenser, and an expansion device. Air at a given state and mass flow rate enters the evaporator. Also at the condenser, air enters at a given state and mass flow rate. The expansion device regulates the mass flow rate of refrigerant in order to maintain the set degree of refrigerant superheating. The compressor (either with fixed or variable speed drive) runs at a given compressor speed. The question is what the steady operating state of the vapor compression machine will be under the given operating conditions.

## Configuration of the Vapor Compression Machine

To configure a single-stage vapor compression machine, we first need to define compressor, evaporator and condenser.

**Compressor**<br>
Based on preliminary calculations (see the first notebook of this series), a variable speed compressor with refrigerant R134a has been selected using selection software of the compressor manufacturer. The polynomial coefficients of the functions describing the performance quantities of this compressor were saved in a csv-file.   

First, define the refrigerant:

In [4]:
R134a = Fluid('R134a')

Now we can define the compressor:

In [5]:
compressor = VariableSpeedCompressor(
    coeff_file=Path("compressor_data/VTZ038-G_R134a.csv"),
    refrigerant_type=R134a,
    units={'m_dot': 'kg / hr', 'speed': '1 / s'}
)

**Evaporator**<br>
We will use the same air evaporator defined in notebook no. 2 (plain fin-tube counterflow evaporator).

In [6]:
evaporator = PlainFinTubeCounterFlowAirEvaporator(
    W_fro=Q_(0.731, 'm'),
    H_fro=Q_(0.244, 'm'),
    N_rows=2,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm')
)

**Condenser**<br>
We will use the same air condenser defined in notebook no. 3 (plain fin-tube counterflow condenser).

In [7]:
condenser = PlainFinTubeCounterFlowAirCondenser(
    W_fro=Q_(0.909, 'm'),
    H_fro=Q_(0.303, 'm'),
    N_rows=4,
    S_trv=Q_(25.4, 'mm'),         
    S_lon=Q_(22.0, 'mm'),         
    D_int=Q_(8.422, 'mm'),        
    D_ext=Q_(10.2, 'mm'),         
    t_fin=Q_(0.3302, 'mm'),       
    N_fin=1 / Q_(3.175, 'mm')    
)

**Vapor compression machine**<br>
The creation of the vapor compression machine is straightforward. First, you pass the evaporator, condenser, and compressor model to the constructor. Then, you specify the refrigerant and the degree of refrigerant superheating set on the expansion device:

In [8]:
machine = SingleStageVaporCompressionMachine(
    evaporator, condenser, compressor, 
    R134a, dT_sh=Q_(10, 'K')
)

## Specify Known Operating Conditions and Determine Steady-State Machine Operation

The following inputs will unambiguously determine the operation of the vapor compression machine:
- the mass flow rate of air through the evaporator
- the mass flow rate of air through the condenser
- the air state at the evaporator inlet
- the air state at the condenser inlet
- the compressor speed (only in case of a variable speed compressor)

To determine the steady-state operation of the machine, an iterative solving technique is used inspired by the method explained in Chapter 14 of the book *Stoecker, W. F., & Jones, J. W. (1982). Refrigeration and air conditioning. McGraw-Hill International Editions, Mechanical Technology Series.* 
The implemented rating routine uses a minimization algorithm from scipy (`scipy.optimize.minimize`) to determine the evaporation and condensation temperature for which the deviation is minimal between the refrigerant mass flow rate according to the compressor model, and the refrigerant mass flow rate according to the evaporator model (i.e., the mass flow rate let through by the expansion device to maintain the set degree of refrigerant superheating at the outlet of the evaporator). The routine needs initial guesses for the evaporation and condensing temperature to start. If the user does not provide these initial guesses, the rating routine will guess the initial values by itself. However, it may happen that no solution can be found with one set of initial guesses, but that a solution is found with another set.

In [9]:
output = machine.rate(
    evp_air_in=HumidAir(Tdb=Q_(24.0, 'degC'), RH=Q_(50, 'pct')),
    evp_air_m_dot=Q_(1500, 'kg / hr'),
    cnd_air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    cnd_air_m_dot=Q_(2233.861, 'kg / hr'),
    n_cmp=Q_(4000, '1 / min')
)

[24844 | hvac.vapor_compression.machine | INFO] Starting machine operation analysis...
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Try with: T_evp = 4.000 °C, T_cnd = 55.000 °C
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate from compressor = 95.109 kg/h
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering condenser with T = 98.112 °C, P = 14.915 bar
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving condenser with T = 42.899 °C, P = 14.915 bar
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering evaporator with T = 4.000 °C, P = 3.377 bar, x = 28.268 %
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving evaporator with T = 14.000 °C, P = 3.377 bar
[24844 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate through evaporator = 106.876 kg/h
[24844 | hvac.vapor_compression.machine | INFO]

In [11]:
print(output.to_text())

evp_air_m_dot = 1500.000 kg/h
cnd_air_m_dot = 2233.861 kg/h
evp_air_in = 24.000 °C DB, 9.340 g/kg AH (50 % RH)
cnd_air_in = 35.000 °C DB, 10.590 g/kg AH (30 % RH)
n_cmp = 4000.000 1/min
dT_sh = 10.000 K
evp_air_out = 17.484 °C DB, 8.131 g/kg AH (65 % RH)
cnd_air_out = 44.229 °C DB, 10.590 g/kg AH (18 % RH)
evp_Q_dot = 4.054 kW
cnd_Q_dot = 5.875 kW
cmp_W_dot = 1.821 kW
rfg_m_dot = 96.242 kg/h
COP = 3.227 frac
EER = 2.227 frac
evp_eps = 0.3294 frac
cnd_eps = 0.3207 frac
T_evp = 5.123 °C
P_evp = 3.512 bar
T_cnd = 60.240 °C
P_cnd = 16.914 bar
dT_sc = 18.402 K
suction_gas = 15.123 °C, 410.693 kJ/kg, 3.512 bar
discharge_gas = 103.615 °C, 478.798 kJ/kg, 16.914 bar
liquid = 41.838 °C, 259.032 kJ/kg, 16.914 bar
mixture = 5.123 °C, 259.032 kJ/kg, 3.512 bar, 27 %
